In [11]:
# load spark DAGs 
import pandas as pd

df = pd.read_csv('/local0/Kariz/expriments/simulator/multidag/config/multi_dag_workload.csv')
df.columns

Index(['app_completion_time', 'app_dag', 'app_id', 'app_name',
       'app_start_time', 'bw', 'ds', 'framework', 'job_completion_time',
       'job_id', 'job_stage_ids', 'job_submission_time', 'name', 'numTasks',
       'outputBytes', 'random', 'stage_completion_time', 'stage_id',
       'stage_input_bytes', 'stage_queue_time', 'stage_rdds', 'stage_run_time',
       'stage_shuffle_read_bytes', 'stage_shuffle_write_bytes',
       'stage_start_time', 'stage_submission_time', 'stride', 'app_runtime',
       'feature', 'tables', 'prefetched_blocks', 'prefetched_bytes', 'bw_i',
       'sequentials'],
      dtype='object')

In [43]:
sptpath = '/local0/Kariz/expriments/simulator/multidag/spark.tpch.template'

def build_graph_from_string(dag_str):
    meta = dag_str.split('\n')[0]
    g_name, g_index = meta.split('Q')[1].split(',')
    g_elements = dag_str.split('\n')[1:]
    g_queuetime = 0
    print(meta, g_index, g_name)
    if len(sq_df[(sq_df['query']==int(g_name)) & (sq_df['index']==int(g_index))]['id'].values) == 0:
        return -1
    g_id = sq_df[(sq_df['query']==int(g_name)) & (sq_df['index']==int(g_index))]['id'].values[0]
    print('query', g_name, 'index', g_index, 'id', g_id)
    
    g = gt.Graph(directed=True)
    g.gp['name'] = g.new_graph_property("string", 'Q%s,%s'%(g_name,g_index))
    g.gp['id'] = g.new_graph_property("int", str(g_id))
    g.gp['queue_time'] = g.new_graph_property("int", g_queuetime)
    g.gp['cur_stage'] = g.new_graph_property("int", -1)
    status = g.new_vertex_property("int")
    inputs = g.new_vertex_property("string")
    color = g.new_vertex_property("string")
    ops = g.new_vertex_property("string")
    vids = g.new_vertex_property("string")
    sv_gv = {}
    # build vertices
    for el in g_elements[:-1]:
        if el.startswith('v'):
           # print(el)
            vid, operation, inputs_str = el.split(',')[1:]
            v = g.add_vertex()
            vids[v] = vid
            sv_gv[vid] = v
            inputs[v] = inputs_str #build_input_format(inputs_str)
            ops[v] = operation if operation else 'skipped'
            color[v] = '#fb8072' if len(inputs[v]) > 0 else '#bdbdbd' if ops[v] == 'skipped' else '#80b1d3'
            #if g_name == '11' and g_index=='1':
            #    print(v, vids[v], inputs[v])
        
    # build edges
    for el in g_elements[:-1]:
        if el.startswith('e'):
            #if g_name == '11' and g_index=='1' :
            #    print(el)
            v_src, v_dest = el.split(',')[1:]
            e = g.add_edge(sv_gv[v_src], sv_gv[v_dest])

    g.vp['vids'] = vids
    g.vp['color'] = color
    g.vp['tables'] = inputs
    g.vp['status'] = status
    g.vp['feature'] = ops
    return g

dm = df.groupby('app_id')['name', 'app_dag'].agg('max').reset_index()
pd.set_option("display.max_colwidth",800)

string_to_query=[]
n_uniques = 0
for i in range(1,23):
    for j, dag_str in enumerate(dm[dm['name']=='tpch%d'%(i)].groupby('app_dag').agg('max').reset_index()['app_dag'].values):
        n_uniques += 1
        string_to_query.append({'query': i, 'dag': dag_str, 'index': j, 'id': n_uniques})

sq_df = pd.DataFrame(string_to_query)


graphs_pool = {}
with open(sptpath, 'r') as fd:
    dag_strs = fd.read().split('#')[1:]
    for index, dag_str in enumerate(dag_strs):
        g = build_graph_from_string(dag_str)
        if isinstance(g, int):
            continue
        graphs_pool[g.gp.id] = g
        if index ==43:
            break
            
print('Total number of graphs in the graph pool', len(graphs_pool))

t      TPCH_Q1,0 0 1
query 1 index 0 id 1
t      TPCH_Q2,0 0 2
query 2 index 0 id 2
t      TPCH_Q2,1 1 2
t      TPCH_Q2,2 2 2
t      TPCH_Q2,3 3 2
t      TPCH_Q3,0 0 3
query 3 index 0 id 3
t      TPCH_Q4,0 0 4
query 4 index 0 id 4
t      TPCH_Q5,0 0 5
query 5 index 0 id 5
t      TPCH_Q5,1 1 5
query 5 index 1 id 6
t      TPCH_Q5,2 2 5
t      TPCH_Q6,0 0 6
query 6 index 0 id 7
t	TPCH_Q7,0 0 7
query 7 index 0 id 8
t      TPCH_Q7,1 1 7
t      TPCH_Q7,2 2 7
t      TPCH_Q8,0 0 8
query 8 index 0 id 9
t	TPCH_Q9,0 0 9
query 9 index 0 id 10
t      TPCH_Q9,1 1 9
t	TPCH_Q10,0 0 10
query 10 index 0 id 11
t      TPCH_Q10,1 1 10
query 10 index 1 id 12
t      TPCH_Q10,2 2 10
t	TPCH_Q12,0 0 12
query 12 index 0 id 16
t	TPCH_Q13,0 0 13
query 13 index 0 id 17
t      TPCH_Q14,0 0 14
query 14 index 0 id 18
t      TPCH_Q15,0 0 15
query 15 index 0 id 19
t	TPCH_Q16,0 0 16
query 16 index 0 id 20
t      TPCH_Q16,1 1 16
t	TPCH_Q17,0 0 17
query 17 index 0 id 21
t      TPCH_Q17,1 1 17
t      TPCH_Q17,2 2 17
t	TPCH_

In [172]:
queries = df.groupby('app_id')['name'].agg('max').values
queries.reshape(10,10)

array([['tpch13', 'tpch11', 'tpch4', 'tpch20', 'tpch17', 'tpch15',
        'tpch10', 'tpch9', 'tpch16', 'tpch4'],
       ['tpch20', 'tpch19', 'tpch1', 'tpch3', 'tpch10', 'tpch22',
        'tpch11', 'tpch6', 'tpch2', 'tpch22'],
       ['tpch22', 'tpch11', 'tpch7', 'tpch21', 'tpch5', 'tpch17',
        'tpch14', 'tpch12', 'tpch4', 'tpch9'],
       ['tpch10', 'tpch18', 'tpch3', 'tpch11', 'tpch21', 'tpch1',
        'tpch17', 'tpch4', 'tpch6', 'tpch10'],
       ['tpch18', 'tpch3', 'tpch16', 'tpch20', 'tpch15', 'tpch2',
        'tpch21', 'tpch5', 'tpch12', 'tpch4'],
       ['tpch9', 'tpch22', 'tpch8', 'tpch6', 'tpch20', 'tpch1', 'tpch5',
        'tpch10', 'tpch11', 'tpch4'],
       ['tpch18', 'tpch22', 'tpch8', 'tpch14', 'tpch11', 'tpch12',
        'tpch9', 'tpch18', 'tpch7', 'tpch8'],
       ['tpch14', 'tpch15', 'tpch16', 'tpch5', 'tpch4', 'tpch8',
        'tpch13', 'tpch6', 'tpch14', 'tpch22'],
       ['tpch2', 'tpch12', 'tpch3', 'tpch18', 'tpch8', 'tpch1', 'tpch13',
        'tpch10', 'tpch

In [190]:
import random 
datasets = ['64G', '32G', '20G']
version = range(0, 10)

n_iteration = 10
n_concurrent = 10

test_datasets = []

for it in range(0, 10):
    for con in range(0, 10):
        test_datasets.append(random.randint(0, 10))
        

In [191]:
import graph_tool.all as gt
import numpy as np

apps = df.groupby('app_id')

convert = lambda a: (int(a[0]) if a[0] != '' else -1, int(a[1]))

i = 0
it = -1
input_dirs = [] 
input_names = []
workload_str = ''
for index, app in apps:
    app_str = app['app_dag'].max()
    ds = app['ds'].max()
    gid = sq_df[sq_df['dag'] == app_str]['id'].values[0]
    g = graphs_pool[gid]
    if it != i//10:
        it = i//10
        workload_str += ("%" + 'r,%d,%d\n'%(it,10))
    ind = i%10
    workload_str += ('#t,%d%d,%s\n'%(it,ind,gid)) 
    
    #print(ds, test_datasets[i], r, ind)
    r = random.randint(0, 9)
    input_dirs.append('%s_%d'%(ds,r))
    
    for v in g.vertices():
        if g.vp.feature[v] != 'skipped':
            t_compute = app[app['stage_id'] == int(g.vp.vids[v])]['stage_run_time'].values[0]
            
            app[app['stage_id'] == int(g.vp.vids[v])]['tables'].fillna('_', inplace=True)
            inv = ''
            if not app[app['stage_id'] == int(g.vp.vids[v])]['tables'].isnull().values[0]:
                inv = 's_%s_%s_%d'%(app[app['stage_id'] == int(g.vp.vids[v])]['tables'].values[0],
                                    ds, r)
                input_names.append('%s_%s_%d'%(app[app['stage_id'] == int(g.vp.vids[v])]['tables'].values[0],
                                    ds, r))
            workload_str += ('v,%d,%s,%s,%d,%d\n'%(int(g.vp.vids[v]),g.vp.feature[v],inv,
                                            int(t_compute), 1))
    for e in g.edges():
        if g.vp.feature[e.source()] != 'skipped' and g.vp.feature[e.target()] != 'skipped':
            workload_str += ('e,%s,%s\n'%(g.vp.vids[e.source()],g.vp.vids[e.target()]))
    
    i = i + 1

print(workload_str)


%r,0,10
#t,00,17
v,0,SCAN:FILTER,s_orders_20G_0,10,1
v,1,SCAN,s_customer_20G_0,5,1
v,2,SORT:SORTMERGEJOIN:HASHAGGREGATE,,5,1
v,3,HASHAGGREGATE,,2,1
v,4,HASHAGGREGATE,,1,1
v,9,HASHAGGREGATE,,1,1
v,10,SORT,,1,1
e,0,2
e,1,2
e,2,3
e,3,4
e,4,9
e,9,10
#t,01,15
v,0,SCAN:FILTER,s_nation_20G_1,4,1
v,1,SCAN,s_supplier_20G_1,5,1
v,2,SORT:SORTMERGEJOIN,,2,1
v,3,SCAN,s_partsupp_20G_1,7,1
v,4,SORT:SORTMERGEJOIN:HASHAGGREGATE,,6,1
v,5,HASHAGGREGATE,,0,1
v,8,SCAN,s_partsupp_20G_1,3,1
v,10,SORT:SORTMERGEJOIN:HASHAGGREGATE,,4,1
v,11,HASHAGGREGATE,,3,1
v,17,HASHAGGREGATE,,1,1
v,18,SORT,,0,1
e,0,2
e,1,2
e,2,4
e,2,10
e,3,4
e,4,5
e,5,17
e,8,10
e,10,11
e,11,17
e,17,18
#t,02,4
v,0,SCAN:FILTER:HASHAGGREGATE,s_orders_20G_7,50,1
v,1,SCAN:FILTER,s_lineitem_20G_7,123,1
v,2,SORT:SORTMERGEJOIN:HASHAGGREGATE,,3,1
v,3,HASHAGGREGATE,,1,1
v,7,HASHAGGREGATE,,1,1
v,8,SORT,,0,1
e,0,2
e,1,2
e,2,3
e,3,7
e,7,8
#t,03,24
v,0,SCAN:FILTER:HASHAGGREGATE,s_partsupp_20G_2,10,1
v,1,SCAN,s_part_20G_2,9,1
v,2,SORT:SORTMERGEJOIN:HASHAGG

In [192]:
workload_file = '/local0/Kariz/expriments/simulator/multidag/config/synthetic_worload_spark_0.g'
with open(workload_file, 'w') as fd:
    fd.write(workload_str)

In [193]:
tpch_queries = [x for x in zip(input_dirs, queries)]
tpch_queries

[('20G_0', 'tpch13'),
 ('20G_1', 'tpch11'),
 ('20G_7', 'tpch4'),
 ('20G_2', 'tpch20'),
 ('20G_6', 'tpch17'),
 ('20G_7', 'tpch15'),
 ('20G_0', 'tpch10'),
 ('20G_3', 'tpch9'),
 ('20G_8', 'tpch16'),
 ('20G_6', 'tpch4'),
 ('20G_3', 'tpch20'),
 ('20G_2', 'tpch19'),
 ('20G_1', 'tpch1'),
 ('20G_1', 'tpch3'),
 ('32G_9', 'tpch10'),
 ('32G_0', 'tpch22'),
 ('32G_1', 'tpch11'),
 ('32G_2', 'tpch6'),
 ('20G_5', 'tpch2'),
 ('20G_7', 'tpch22'),
 ('20G_4', 'tpch22'),
 ('20G_3', 'tpch11'),
 ('20G_5', 'tpch7'),
 ('20G_8', 'tpch21'),
 ('20G_2', 'tpch5'),
 ('20G_9', 'tpch17'),
 ('20G_0', 'tpch14'),
 ('20G_7', 'tpch12'),
 ('20G_4', 'tpch4'),
 ('32G_3', 'tpch9'),
 ('32G_3', 'tpch10'),
 ('32G_1', 'tpch18'),
 ('32G_7', 'tpch3'),
 ('32G_5', 'tpch11'),
 ('32G_6', 'tpch21'),
 ('32G_6', 'tpch1'),
 ('32G_5', 'tpch17'),
 ('32G_3', 'tpch4'),
 ('64G_0', 'tpch6'),
 ('64G_3', 'tpch10'),
 ('64G_3', 'tpch18'),
 ('64G_0', 'tpch3'),
 ('64G_5', 'tpch16'),
 ('64G_2', 'tpch20'),
 ('64G_0', 'tpch15'),
 ('64G_0', 'tpch2'),
 ('64

In [194]:
# loop over pig tpch queries 
# read graphs and build graph pool 
import graph_tool.all as gt

import sys
sys.path.append('/local0/Kariz/code')
import utils.objectstore as object_store

def build_input_format(inputs_str):
    return ' '.join(inputs_str.split(':')).split()

def load_graph_templates(path):
    graph_skeletons = {}
    with open(path, 'r') as fd:
        graph_strs = fd.read().split('#')[1:]
        for g_str in graph_strs:
            g= build_graph_skeleton(g_str)
            graph_skeletons[g.gp.name] = g
    return graph_skeletons


def build_graph_skeleton(g_str):
    g_elements = g_str.split('\n')
    g_name = g_elements[0].split('\t')[1]
    g_id = 0
    g_queuetime = 0

    g = gt.Graph(directed=True)
    g.gp['name'] = g.new_graph_property("string", g_name)
    g.gp['id'] = g.new_graph_property("string", str(g_id))
    g.gp['queue_time'] = g.new_graph_property("int", g_queuetime)
    g.gp['cur_stage'] = g.new_graph_property("int", -1)
    status = g.new_vertex_property("int")
    inputs = g.new_vertex_property("string")
    cache_runtime = g.new_vertex_property("float")
    remote_runtime = g.new_vertex_property("int")
    color = g.new_vertex_property("string")
    ops = g.new_vertex_property("string")
    vids = g.new_vertex_property("int")
    id_v = {}
    
    # build vertices
    for el in g_elements[1:]:
        if el.startswith('v'):
            vid, operation, inputs_str = el.split(',')[1:]
            v = g.add_vertex()
            vids[v] = int(vid)
            id_v[int(vid)] = v
            inputs[v] = inputs_str #build_input_format(inputs_str)
            #print(inputs_str)
            color[v] = '#fb8072' if len(inputs[v]) > 0 else '#80b1d3'
            cache_runtime[v] = 0
            remote_runtime[v] = 0
            ops[v] = operation if operation else 'SAVE'

    # build edges
    for el in g_elements[1:]:
        if el.startswith('e'):
            v_src, v_dest = el.split(',')[1:]
            e = g.add_edge(id_v[int(v_src)], id_v[int(v_dest)])

    g.vp['id'] = vids
    g.vp['color'] = color
    g.vp['tables'] = inputs
    g.vp['remote_runtime'] = remote_runtime
    g.vp['cache_runtime'] = cache_runtime
    g.vp['status'] = status
    g.vp['feature'] = ops
    print('Load query:', g_name)
    return g

graphs_pool2 = load_graph_templates('/local0/Kariz/expriments/simulator/multidag/%s'%('pig.tpch.template'))

Load query: TPCH_Q12
Load query: TPCH_Q17
Load query: TPCH_Q9
Load query: TPCH_Q4
Load query: TPCH_Q16
Load query: TPCH_Q8
Load query: TPCH_Q7
Load query: TPCH_Q22
Load query: TPCH_Q19
Load query: TPCH_Q6
Load query: TPCH_Q3
Load query: TPCH_Q5
Load query: TPCH_Q15
Load query: TPCH_Q18
Load query: TPCH_Q13
Load query: TPCH_Q10
Load query: TPCH_Q14
Load query: TPCH_Q20
Load query: TPCH_Q21
Load query: TPCH_Q2
Load query: TPCH_Q11
Load query: TPCH_Q1


In [195]:
workload_str = ''

it = -1
for i, x in enumerate(tpch_queries):
    query = 'TPCH_Q' + x[1].replace('tpch', '')
    g = graphs_pool2[query].copy()
    ds = x[0]
    
    if it != i//10:
        it = i//10
        workload_str += ("%" + 'r,%d,%d\n'%(it,10))
    ind = i%10
    workload_str += ('#t,%d%d,%s\n'%(it,ind,gid)) 
    
    
    for v in g.vertices():
        t_compute = random.randint(5, 100)
        g.vp.remote_runtime[v] = t_compute
        g.vp.cache_runtime[v] = 1
        input_str=''
        if len(g.vp.tables[v]) > 0:  
            g.vp.cache_runtime[v] = random.uniform(0.3, 1)
            inputs = g.vp.tables[v].split(':')
            input_str = ':'.join(['p_%s_%s'%(al,ds) for al in inputs])
        workload_str += ('v,%d,%s,%s,%d,%.2f\n'%(g.vp.id[v],g.vp.feature[v],
                                                 input_str,g.vp.remote_runtime[v], 
                                                 g.vp.cache_runtime[v]))
            
    for e in g.edges():
        workload_str += ('e,%d,%d\n'%(g.vp.id[e.source()],g.vp.id[e.target()]))
    #if i ==2:
    #    break 
print(workload_str)   

%r,0,10
#t,00,24
v,0,COGROUP,p_customer_20G_0:p_orders_20G_0,21,0.55
v,1,GROUP_BY:COMBINER,,7,1.00
v,2,SAMPLER,,27,1.00
v,3,ORDER_BY,,65,1.00
e,0,1
e,1,2
e,2,3
#t,01,24
v,0,HASH_JOIN,p_nation_20G_1:p_supplier_20G_1,5,0.83
v,1,HASH_JOIN,p_partsupp_20G_1,24,0.74
v,2,GROUP_BY:COMBINER,,68,1.00
v,3,GROUP_BY:COMBINER,,85,1.00
v,4,SAMPLER,,91,1.00
v,5,ORDER_BY,,7,1.00
e,0,1
e,1,2
e,1,3
e,2,3
e,3,4
e,4,5
#t,02,24
v,0,COGROUP,p_lineitem_20G_7:p_orders_20G_7,62,0.35
v,1,GROUP_BY:COMBINER,,87,1.00
v,2,SAMPLER,,96,1.00
v,3,ORDER_BY,,90,1.00
e,0,1
e,1,2
e,2,3
#t,03,24
v,0,GROUP_BY:COMBINER,p_lineitem_20G_2,45,0.65
v,1,HASH_JOIN,,92,1.00
v,2,DISTINCT,p_part_20G_2,15,0.90
v,3,HASH_JOIN,p_partsupp_20G_2,74,0.58
v,4,DISTINCT,,50,1.00
v,5,HASH_JOIN,,56,1.00
v,6,HASH_JOIN,p_nation_20G_2:p_supplier_20G_2,69,0.92
v,7,SAMPLER,,36,1.00
v,8,ORDER_BY,,9,1.00
e,0,1
e,1,4
e,2,3
e,3,1
e,4,5
e,5,7
e,6,5
e,7,8
#t,04,24
v,0,COGROUP,p_lineitem_20G_6:p_part_20G_6,19,0.35
v,1,GROUP_BY:COMBINER,,72,1.00
e,0,1
#t,05,24


In [197]:
workload_file = '/local0/Kariz/expriments/simulator/multidag/config/synthetic_worload_pig_0.g'
with open(workload_file, 'w') as fd:
    fd.write(workload_str)

In [198]:
names = ''
for name in input_names:
    max_ = int(name.split('_')[1].replace('G', ''))
    size = random.randint(1, max_)
    names += 'p_%s,%d\ns_%s,%d\n%s,%d\n'%(name, size, name, size, name, size)
    

print(names)

p_orders_20G_0,7
s_orders_20G_0,7
orders_20G_0,7
p_customer_20G_0,17
s_customer_20G_0,17
customer_20G_0,17
p_nation_20G_1,15
s_nation_20G_1,15
nation_20G_1,15
p_supplier_20G_1,15
s_supplier_20G_1,15
supplier_20G_1,15
p_partsupp_20G_1,17
s_partsupp_20G_1,17
partsupp_20G_1,17
p_partsupp_20G_1,14
s_partsupp_20G_1,14
partsupp_20G_1,14
p_orders_20G_7,6
s_orders_20G_7,6
orders_20G_7,6
p_lineitem_20G_7,3
s_lineitem_20G_7,3
lineitem_20G_7,3
p_partsupp_20G_2,11
s_partsupp_20G_2,11
partsupp_20G_2,11
p_part_20G_2,2
s_part_20G_2,2
part_20G_2,2
p_lineitem_20G_2,14
s_lineitem_20G_2,14
lineitem_20G_2,14
p_nation_20G_2,1
s_nation_20G_2,1
nation_20G_2,1
p_supplier_20G_2,10
s_supplier_20G_2,10
supplier_20G_2,10
p_part_20G_6,8
s_part_20G_6,8
part_20G_6,8
p_lineitem_20G_6,10
s_lineitem_20G_6,10
lineitem_20G_6,10
p_lineitem_20G_6,2
s_lineitem_20G_6,2
lineitem_20G_6,2
p_lineitem_20G_7,1
s_lineitem_20G_7,1
lineitem_20G_7,1
p_customer_20G_0,19
s_customer_20G_0,19
customer_20G_0,19
p_orders_20G_0,18
s_orders_2

In [199]:
fpath = '/local0/Kariz/expriments/simulator/multidag/config/inputs.csv'
with open(fpath, 'a') as fd:
    fd.write(names)

In [202]:
tnames = {}
for i in input_names:
    if i.split('_')[0] not in tnames:
        tnames[i.split('_')[0]] = 0
        
print(tnames.keys())

dict_keys(['orders', 'customer', 'nation', 'supplier', 'partsupp', 'lineitem', 'part', 'region'])


In [ ]:
import random.